In [2]:
import cv2
import pytesseract

In [3]:
img=cv2.imread("sample.jpeg")
img

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [4]:
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [5]:
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

In [6]:
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

In [7]:
import cv2
import pytesseract
import re

# Function to filter out invalid or incomplete hormone names
def filter_hormones(hormone_list):
    valid_hormones = []
    for hormone in hormone_list:
        # Remove any text that is too short (e.g., less than 3 characters) or has numbers or special characters
        if len(hormone) > 2 and hormone.isalpha():
            valid_hormones.append(hormone.strip())
    return valid_hormones

# Assuming you already have your image processed to get contours

# Dilation and contour extraction (unchanged)
dilation = cv2.dilate(thresh1, rect_kernel, iterations=1)
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()

# List to store recognized hormones
hormones = []

# Looping through the identified contours
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    
    # Drawing a rectangle on copied image
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]

    # Open the file to append the extracted text
    file = open("recognized.txt", "a")
    
    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped)
    
    # Clean up the text (replace newlines with spaces, strip leading/trailing spaces)
    cleaned_text = text.replace("\n", " ").strip()
    
    # Split the cleaned text based on commas and newlines (both)
    hormone_list = re.split(r'[,\n]', cleaned_text)
    
    # Clean up any empty strings or excessive spaces
    hormone_list = [hormone.strip() for hormone in hormone_list if hormone.strip()]
    
    # Filter valid hormones
    valid_hormones = filter_hormones(hormone_list)
    
    # If there are any valid hormone names, append them to the hormones list
    if valid_hormones:
        hormones.extend(valid_hormones)
        file.write(" ".join(valid_hormones) + "\n")
    
    # Close the file after writing
    file.close()

# Print the recognized hormones
print(hormones)


['Relaxin', 'Prolactin', 'Androgens', 'Progesterone', 'Estrogens', 'Erythropoietin', 'Renin', 'Calcitriol', 'Somatostatin', 'Insulin', 'Glucagon', 'Noradrenaline', 'Adrenaline', 'Neuropeptide', 'Glucocorticoids', 'Somatostatin', 'Androgens', 'Histamine', 'Gastrin', 'Thymopoietin', 'PTH', 'Calcitonin', 'Vasopressin', 'Oxytocin', 'MSH', 'FSH', 'TSH', 'Vasopressin', 'Somatostatin', 'Dopamine', 'TRH', 'CRH', 'Melatborn']
